# 1. 학습용 데이터셋 준비

In [ ]:
import pandas as pd

# 파일 경로
phishing_path = "../../dataset/phishing_total-0716_0945.csv"
general_path = "../../dataset/normal_비율맞춘_학습데이터셋.csv"  # 일반 통화 데이터셋 파일명에 맞게 수정

# 데이터 불러오기
phishing_df = pd.read_csv(phishing_path)
general_df = pd.read_csv(general_path)

# is_phishing 컬럼 추가
phishing_df["is_phishing"] = 1
general_df["is_phishing"] = 0

# 컬럼 일치 여부 확인 후, 필요한 컬럼만 맞춰서 선택 (예: speaker, text 등)
# 공통 컬럼으로 정렬 (추가 컬럼이 있어도 무방하되, 학습 시 정제 필요)
common_columns = list(set(phishing_df.columns).intersection(set(general_df.columns)))
phishing_df = phishing_df[common_columns]
general_df = general_df[common_columns]

# 합치기
total_df = pd.concat([phishing_df, general_df], ignore_index=True)

# 결과 저장 (선택)
total_df.to_csv("../../dataset/merged_labeled_data.csv", index=False)

print("✅ is_phishing 컬럼 추가 및 병합 완료!")

# 2. 토큰화 및 키워드 추출할 데이터셋 준비

## 2.1.토큰화 할 데이터셋 적절하게 다듬기

### 2.1.1 보이스피싱데이터의 phishing_type 세금환급형 -> 기관사칭형으로 바꾸기

In [2]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("../../dataset/phishing_total.csv")

# phishing_type 컬럼에서 '세금환급형'을 '기관사칭형'으로 변경
df["phishing_type"] = df["phishing_type"].replace("세금환급형", "기관사칭형")

# 저장
df.to_csv("../../dataset/phishing_total_세금기관변환_0716.csv", index=False)

### 2.1.2 보이스피싱 데이터셋의 file_name을 file_id로 바꾸고 is_phishing컬럼을 추가하여 값을 1로 추가 후 같은 file_id의 모든 text 합치기.

In [ ]:
# file_name → file_id 로 컬럼명 변경
df.rename(columns={"file_name": "file_id"}, inplace=True)

# is_phishing 컬럼 추가 (값은 모두 1)
df["is_phishing"] = 1

# speaker 컬럼 삭제
df = df.drop(columns=["speaker"])

# 같은 file_id 별로 text 합치기 (그 외 컬럼은 첫 번째 값 사용)
aggregated_df = (
    df.groupby("file_id")
    .agg(
        {
            "phishing_type": "first",  # 그룹 내 첫번째 phishing_type 사용
            "text": " ".join,  # 텍스트를 공백으로 연결
            "is_phishing": "first",
        }
    )
    .reset_index()
)
print(df.head())

        file_id phishing_type  speaker  \
0  phishing_000       가족지인사칭형        0   
1  phishing_000       가족지인사칭형        1   
2  phishing_000       가족지인사칭형        0   
3  phishing_000       가족지인사칭형        1   
4  phishing_000       가족지인사칭형        0   

                                     text  is_phishing  
0                      여보세요, OOO야. 나 아빠야.            1  
1                       아빠? 목소리가 좀 이상한데요.            1  
2  아빠 핸드폰이 고장 나서 친구 폰으로 걸었어. 급하게 돈이 필요해서.            1  
3                         무슨 일인데 그렇게 급해요?            1  
4      교통사고가 나서 병원비가 필요해. 믿고 바로 보내줄 수 있지?            1  


In [4]:
df = df.drop(columns=["speaker"])

In [5]:
df

,file_id,phishing_type,text,is_phishing
0,phishing_000,가족지인사칭형,"여보세요, OOO야. 나 아빠야.",1
1,phishing_000,가족지인사칭형,아빠? 목소리가 좀 이상한데요.,1
2,phishing_000,가족지인사칭형,아빠 핸드폰이 고장 나서 친구 폰으로 걸었어. 급하게 돈이 필요해서.,1
3,phishing_000,가족지인사칭형,무슨 일인데 그렇게 급해요?,1
4,phishing_000,가족지인사칭형,교통사고가 나서 병원비가 필요해. 믿고 바로 보내줄 수 있지?,1
...,...,...,...,...
44514,phishing_2045,대출빙자형,지금바로 통화 가능하신가요?,1
44515,phishing_2045,대출빙자형,네 지금 업무시간이라서 원래 휴대폰 못만지는데 바로 연락 받겠습니다,1
44516,phishing_2045,대출빙자형,네 알겠습니다,1
44517,phishing_2045,대출빙자형,네,1


In [6]:
aggregated_df.to_csv("phishing_total_grouped.csv", index=False)

### 2.2.1 일반통화 데이터셋 토큰화시킬 수 있는 상태까지 전처리하기

In [7]:
df_normal = pd.read_csv("../../dataset/normal_비율맞춘_학습데이터셋.csv")
# is_phishing 컬럼 추가 (값은 모두 0)
df_normal["is_phishing"] = 0
# file_id 컬럼 추가 (file_name에서 file_id로 변경)
df_normal.rename(columns={"file_name": "file_id"}, inplace=True)
# speaker 컬럼 삭제
df_normal = df_normal.drop(columns=["speaker"])
# 같은 file_id 별로 text 합치기 (그 외 컬럼은 첫 번째 값 사용)
aggregated_normal_df = (
    df_normal.groupby("file_id")
    .agg(
        {
            "text": " ".join,  # 텍스트를 공백으로 연결
            "is_phishing": "first",
        }
    )
    .reset_index()
)
# 저장
df_normal.to_csv("../../dataset/normal_비율맞춘_학습데이터셋_완성본.csv", index=False)



## 2.3 두 데이터프레임 합치기

In [8]:
# 두 데이터프레임 합치기
merged_df = pd.concat([aggregated_df, aggregated_normal_df], ignore_index=True)
merged_df.to_csv("../../dataset/merged_labeled_data.csv", index=False)

In [9]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8047 entries, 0 to 8046
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   file_id        8047 non-null   object
 1   phishing_type  2047 non-null   object
 2   text           8047 non-null   object
 3   is_phishing    8047 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 251.6+ KB
